In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [19]:
# 사설 사이트 커넥션이 끊어지는 사이트 에서 쓰기에는 selinium 으로 바꿔서 쓰면 좋다
# 셀리늄은 웹드라이버를 만들어서 알아서 사람인척하면서 다닌다. 
# 근데 공공기관은 별로 공격을 당하지 않아서 그냥 BeautifulSoup 쓰면 된다.

In [20]:
#크롤링 정보를 저장할 데이터프레임 선언
data = pd.DataFrame(columns = ['스테이션명','위치','상태정보','위도','경도'])

# 크롤링 할 타슈 사이트 https://www.tashu.or.kr/userStationAction.do?process=list

url_tashu = "http://www.tashu.or.kr/userStationAction.do?process=list" #https 는 보안서버라서 http로 해야 된다.

# 페이지 이동 시 추가로 적어줘야 되는 url
url_add = "&page="

# 전체 페이지 수
total_page = 27

# 27 page에 대한 정보를 추출하는 for문
for index in range(1,total_page+1): #total_page에서 +1을 꼭 해줘야 한다 알지?
  if index == 1 : #첫번째 페이지라면?
    url = url_tashu #첫번째 페이지
  else: #첫번째 페이지가 아니라면? 
    url = url_tashu + url_add + str(index) # 페이지 정보 추가
  
  # url에서 HTML 추출
  html = urlopen(url)
  
  # HTML을 parsing하여 변환
  bsObject = BeautifulSoup(html, "html.parser", from_encoding='utf-8')

  # history_table이라는 class를 가지 table만 찾기
  table = bsObject.find_all('table', {'class','history_table'})

  # table 내에서 tr 찾기
  tr = table[0].find_all('tr')

  #첫 tr은 contents가 아니라 데이터 정보를 의미하여 제외 [1:tr수까지]
  tr = tr[1:len(tr)]

  # 한 페이지의 정보를 크롤링
  for index_tr in range(0,len(tr)):
    td = tr[index_tr].find_all('td')

    # 장소
    station = td[0].text.split('.')[1] # 1. 무역전시관입구(택시승강장)

    # 위치
    location = td[1].text

    # 상태
    condition = td[2].text

    # 좌표추출
    geo = td[3].a.attrs['href'].split('(')[1]  # '36.374580', '127.386131');" 추출

    # 위도, 경도 추출을 위한 데이터 클렌징
    geo = re.sub('[;)]','',geo) # 대괄호 안에 있는 것들을 다 빈칸으로 만들어버려라=즉,;하고)를 지워라 = '36.374389', '127.392375');" -> '36.374389', '127.392375'
    detail_location = geo.replace("'","")

    # 위도 Latitude
    lat = detail_location.split(',')[0]

    # 경도 Longitude
    lon = detail_location.split(',')[1]

    # data에 크롤링 결과 저장
    data = data.append({'스테이션명':station,
                        '위치': location,
                        '상태정보':condition,
                        '위도':lat,
                        '경도':lon},
                        ignore_index=True)
    print('Complete of' + str(index_tr+1) + 'information with' + str(index) + 'Page')
    
  # Crawling 막히는 것 방지
  time.sleep(1)
print('------------------------------------------------------------------------')
print(data)

Complete of1information with1Page
Complete of2information with1Page
Complete of3information with1Page
Complete of4information with1Page
Complete of5information with1Page
Complete of6information with1Page
Complete of7information with1Page
Complete of8information with1Page
Complete of9information with1Page
Complete of10information with1Page
Complete of1information with2Page
Complete of2information with2Page
Complete of3information with2Page
Complete of4information with2Page
Complete of5information with2Page
Complete of6information with2Page
Complete of7information with2Page
Complete of8information with2Page
Complete of9information with2Page
Complete of10information with2Page
Complete of1information with3Page
Complete of2information with3Page
Complete of3information with3Page
Complete of4information with3Page
Complete of5information with3Page
Complete of6information with3Page
Complete of7information with3Page
Complete of8information with3Page
Complete of9information with3Page
Complete of1

크롤링 저장하기

In [21]:
data.to_csv('TASHU.csv',encoding='utf-8-sig')